In [ ]:
# default_exp data

# Data

> Load transaction data into a usable format

In [ ]:
#hide
from nbdev.showdoc import *

The data module can be used to load in transaction data sets from multiple sources into a format that the gamba library can use.

## Reading CSV Files

This can be done using the `read_csv` method, which is a wrapper around the [pandas library's method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) of the same name.

In [ ]:
#export
def read_csv(file, parse_dates=[], index_col=None, delimiter=",", dummy_data=False):

    df = pd.read_csv(
        file, parse_dates=parse_dates, index_col=index_col, delimiter=delimiter
    )

    return df

This can be used to read a regular CSV file as you'd expect, but can also be used for tab-separated files - as some of [the transparency project](http://www.thetransparencyproject.org/download_index.php)'s data sets are.

## Setting Column Names

The gamba library's methods expect a dataframe with certain column names. The most important task after loading data as a [pandas dataframe]() is to set these names according to the type of data the column contains. These names should match the following table;

| name      | description                                          |
|-----------|------------------------------------------------------|
| player_id | a unique identifier for each player                  |
| bet_size  | the size of the bet (in raw currency form, e.g. USD) |
| bet_time  | the                                                  |

## Final Checks

It's good practice to check that your column names match those used by the gamba library, and make sure that no extra columns exist. The `check_data` method below can be given the dataframe, and it will raise an error if anything isn't as it should be;

In [ ]:
#export
def check_data(dataframe):
    
    column_names = dataframe.columns
    
    print(column_names)
    

    